In [2]:
!pip install schedule
!pip install bs4
!pip install selenium
!pip install webdriver-manager



[notice] A new release of pip is available: 23.0 -> 24.2
[notice] To update, run: C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 24.2
[notice] To update, run: C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 24.2
[notice] To update, run: C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0 -> 24.2
[notice] To update, run: C:\Users\admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
#imports
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
import schedule

In [4]:
# Scraper Function
def scrape_moralis():
    url = 'https://moralis.com'  # Change to the correct page URL if necessary
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Initialize lists to store extracted data
        tokens = []
        hype_scores = []
        rois = []
        fdvs = []
        coin_ages = []
        prices = []

        # Example selectors: You need to inspect the page and change them to match the HTML structure
        for row in soup.select('table tbody tr'):  # Adjust the selector for rows of the table
            token = row.select_one('.token-name').text.strip()  # Example, adjust based on HTML structure
            hype_score = row.select_one('.hype-score').text.strip()
            roi = row.select_one('.roi').text.strip()
            fdv = row.select_one('.fdv').text.strip()
            coin_age = row.select_one('.coin-age').text.strip()
            price = row.select_one('.price').text.strip()
            
            # Append the values to lists
            tokens.append(token)
            hype_scores.append(hype_score)
            rois.append(roi)
            fdvs.append(fdv)
            coin_ages.append(coin_age)
            prices.append(price)
        
        # Create DataFrame
        df = pd.DataFrame({
            'Token': tokens,
            'Hype Score': hype_scores,
            'ROI': rois,
            'FDV': fdvs,
            'Coin Age': coin_ages,
            'Price': prices
        })
        
        # Display the dataframe in the notebook for checking
        display(df)

        # Optionally save to CSV for each run (overwrites each time)
        df.to_csv('crypto_data.csv', index=False)
        
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")

In [5]:
# Scheduling the scraping every 5 minutes (for demonstration, can be changed to 1 minute)
schedule.every(1).minutes.do(scrape_moralis)

Every 1 minute do scrape_moralis() (last run: [never], next run: 2024-10-13 14:44:51)

In [8]:
# Running the scheduled task in a notebook
# Due to the nature of Jupyter, we will simulate running it periodically in cells, rather than a constant loop.
def run_scheduled_tasks():
    schedule.run_pending()
    time.sleep(1)


In [9]:
run_scheduled_tasks()

Failed to retrieve data. Status code: 403


In [30]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Path to ChromeDriver executable
chrome_driver_path = 'C:\\Users\\admin\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe'  # Make sure the path is correct and properly formatted

# Initialize the ChromeDriver using Selenium
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# Navigate to the target website
driver.get('https://moralis.com/top/trending/tokens/solana?ref=yzqwnmj&tapid=&program=moralis-money&gad_source=1&gclid=CjwKCAjw3624BhBAEiwAkxgTOiS0frtwlDpIsvY7_LUE_2Ul4Blo1c5hAWS2BoqXmZISgcBgN2oQUBoCR-0QAvD_BwE')

# Wait for the page to fully load (adjust the time if needed)
time.sleep(5)

# Use explicit wait to ensure the table rows are available
wait = WebDriverWait(driver, 20)  # Wait up to 20 seconds
rows = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'tbody tr')))  # Use 'tbody tr' for table rows

# Lists to store the extracted data
tokens = []
hype_scores = []
rois = []
prices = []
fdvs = []
coin_ages = []
volumes = []
followers = []  # Correct list for followers

# Extract the data from each row
for row in rows:
    try:
        # Extracting Token Name
        try:
            token_element = row.find_element(By.CSS_SELECTOR, 'td.TrendingPageUITableBody_fieldToken__9qzZ5')
            a_tag = token_element.find_element(By.CSS_SELECTOR, 'a.TrendingPageUITableBody_tokenInfo___H_mg')
            span_token_info = a_tag.find_element(By.CSS_SELECTOR, 'span.TrendingPageUITableBody_tokenInfoBox__eKdh3')
            token_name_element = span_token_info.find_element(By.CSS_SELECTOR, 'span.TrendingPageUITableBody_tokenInfoBoxTitleName__3srIY.TrendingPageUITableBody_tokenInfoBoxTitleNameDesktop__25Dtv.body-16-medium')
            token_name = token_name_element.text
        except Exception as e:
            print(f"Error extracting token name: {e}")
            token_name = None
        tokens.append(token_name)
        
        # Extracting Hype Score
        try:
            hype_score_element = row.find_element(By.CSS_SELECTOR, 'td span.TrendingPageUITableBody_batteryChartValueShowUser__TxxV_')
            hype_score = hype_score_element.text
        except Exception as e:
            print(f"Error extracting hype score: {e}")
            hype_score = None
        hype_scores.append(hype_score)

        # Extracting ROI
        try:
            roi_element = row.find_element(By.CSS_SELECTOR, 'td span.body-16-medium.TrendingPageUITableBody_positive__n_l46')
            roi = roi_element.text
        except Exception as e:
            print(f"Error extracting ROI: {e}")
            roi = None
        rois.append(roi)

        # Extracting Price
        try:
            price_element = row.find_element(By.CSS_SELECTOR, 'td span.caption-12-medium.TrendingPageUITableBody_tokenInfoBoxValue__xwWRH')
            price = price_element.text
        except Exception as e:
            print(f"Error extracting price: {e}")
            price = None
        prices.append(price)

        # Extracting FDV
        try:
            fdv_element = row.find_element(By.CSS_SELECTOR, 'td span.body-16-medium.TrendingPageUITableBody_tokenInfoBoxValue__xwWRH')
            fdv = fdv_element.text
        except Exception as e:
            print(f"Error extracting FDV: {e}")
            fdv = None
        fdvs.append(fdv)

        # Extracting Coin Age
        try:
            coin_age_element = row.find_element(By.CSS_SELECTOR, 'td span.body-16-medium')
            coin_age = coin_age_element.text
        except Exception as e:
            print(f"Error extracting coin age: {e}")
            coin_age = None
        coin_ages.append(coin_age)

        # Extracting Volume
        try:
            volume_element = row.find_element(By.CSS_SELECTOR, 'td div.styles_tokenMetric__IbP32')
            volume = volume_element.text
        except Exception as e:
            print(f"Error extracting volume: {e}")
            volume = None
        volumes.append(volume)

        # Extracting Followers
        try:
            followers_element = row.find_element(By.CSS_SELECTOR, 'td span.body-16-medium')
            follower = followers_element.text
        except Exception as e:
            print(f"Error extracting followers: {e}")
            follower = None
        followers.append(follower)

    except Exception as e:
        print(f"Error processing row: {e}")

#close 
driver.quit()

# Ensure all lists have the same length
min_length = min(len(tokens), len(hype_scores), len(rois), len(prices), len(fdvs), len(coin_ages), len(volumes), len(followers))

# Adjust lists to ensure they are the same length
tokens = tokens[:min_length]
hype_scores = hype_scores[:min_length]
rois = rois[:min_length]
prices = prices[:min_length]
fdvs = fdvs[:min_length]
coin_ages = coin_ages[:min_length]
volumes = volumes[:min_length]
followers = followers[:min_length]

# Now create the DataFrame with the same-length lists
df = pd.DataFrame({
    'Token': tokens,
    'Hype Score': hype_scores,
    'ROI': rois,
    'Trending Since': prices,
    'Price (24h)': fdvs,
    'Coin Age': coin_ages,
    'Volume': volumes,
    'Followers': followers
})

# Display the data
print(df)


   Token Hype Score      ROI          Trending Since Price (24h) Coin Age  \
0               63%  +23.36%   since trending 8h ago     +23.36%            
1               61%   +7.97%   since trending 2h ago      +7.97%            
2               60%  +15.56%  since trending 10m ago          0%            
3               60%   +0.68%  since trending 10m ago      +0.68%            
4               58%   +0.16%  since trending 20m ago      +0.16%            
5               57%   +7.83%   since trending 3h ago      +7.83%            
6               57%  +21.89%   since trending 4h ago     +21.89%            
7               57%  +11.35%   since trending 4h ago     +11.35%            
8               57%   +2.37%          found just now          0%            
9               56%  +14.96%   since trending 1h ago     +14.96%            
10              56%   +0.10%   since trending 1h ago          0%            
11              56%   +1.51%   since trending 3h ago      +1.51%            

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Path to ChromeDriver executable (update this path as necessary)
chrome_driver_path = 'C:/webdrivers/chromedriver.exe'

# Initialize the ChromeDriver using Selenium
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# Navigate to the target website
driver.get('https://moralis.com/top/trending/tokens/solana')

# Wait for the page to fully load
time.sleep(5)

# Use explicit wait to ensure the table rows are available
wait = WebDriverWait(driver, 10)  # Wait up to 10 seconds for elements to appear
rows = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.TrendingPageUITableBody_fieldToken__9qZ5')))  # Adjust selector

tokens = []
hype_scores = []
rois = []
fdvs = []
coin_ages = []
prices = []

# Extract the data from each row
for row in rows:
    token = row.find_element(By.CSS_SELECTOR, '.TrendingPageUITableBody_tokenInfo__h__Mp span').text  # Token name
    hype_score = row.find_element(By.CSS_SELECTOR, '.caption-14-medium').text  # Hype score
    # Adjust these selectors as per the HTML structure for ROI, FDV, Coin Age, Price
    # Since I do not see the exact classes for these fields in the screenshot, update based on inspection
    
    # Add to lists
    tokens.append(token)
    hype_scores.append(hype_score)
    # Add extracted data for ROI, FDV, Coin Age, Price

# Create a DataFrame from the extracted data
df = pd.DataFrame({
    'Token': tokens,
    'Hype Score': hype_scores,
    # Add other columns such as ROI, FDV, Coin Age, Price
})

# Display the DataFrame in the notebook
df

# Close the ChromeDriver
driver.quit()


WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home
